<a href="https://colab.research.google.com/github/kerryback/ClassicTests/blob/main/PlotlyExamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#@title Example of portfolio risk and return
import numpy as np
import pandas as pd
import statsmodels.api as sm
import plotly.express as px

try: 
  from jupyter_dash import JupyterDash
  from dash import dcc, html, Output, Input
except :
  !pip install jupyter-dash
  from jupyter_dash import JupyterDash
  from dash import dcc, html, Output, Input

app = JupyterDash(__name__, title='Another Plotly Plot')

app.layout = html.Div([
    dcc.Markdown(''' The plot shows the expected returns and risks of portfolios of two stocks.
    A portfolio is a pair (wt1, wt2) where wt1 denotes the fraction of wealth
    invested in asset 1 and wt2 denotes the fraction of wealth invested in asset 2.
    This plot shows portfolios with wt1 >=0 and wt2 >= 0 (no short sales) and
    wt1+wt2 = 1 (fully invested).  The portfolio risk is lower when the correlation 
    between the asset returns is lower. '''),
    dcc.Dropdown(id='rp1_dropdown',
                 value=6,
                 options=[{'label': 'Expected return of stock 1 = ' + str(rp1)+'%', 'value': rp1}
                          for rp1 in range(4,18,2)]),
    dcc.Dropdown(id='rp2_dropdown',
                 value=14,
                 options=[{'label': 'Expected return of stock 2 = ' + str(rp2)+'%', 'value': rp2}
                          for rp2 in range(4,18,2)]),
    dcc.Dropdown(id='sd1_dropdown',
                 value=20,
                 options=[{'label': 'Standard deviation of stock 1 = ' + str(sd1)+'%', 'value': sd1}
                          for sd1 in range(15,45,5)]),
    dcc.Dropdown(id='sd2_dropdown',
                 value=30,
                 options=[{'label': 'Standard deviation of stock 2 = ' + str(sd2)+'%', 'value': sd2}
                          for sd2 in range(15,45,5)]),
    dcc.Graph(id='myfig'),
])

@app.callback(Output('myfig', 'figure'),
              Input('rp1_dropdown', 'value'),
              Input('rp2_dropdown', 'value'),
              Input('sd1_dropdown', 'value'),
              Input('sd2_dropdown', 'value'))
def myplot(rp1,rp2,sd1,sd2) :
  rps = [rp1,rp2]
  sds = [sd1,sd2]
  df = pd.DataFrame(dtype=float,columns=['correlation','wt1','wt2','mean','stdev','msize'])
  ports = [np.array([w,1-w]) for w in np.linspace(0,1,101)]
  means = rps
  means = [p.T @ means for p in ports]
  df = None
  for corr in np.linspace(-1,1,101) :
    cov = np.array([[sds[0]**2,sds[0]*sds[1]*corr],[sds[0]*sds[1]*corr,sds[1]**2]]).reshape(2,2)
    d = pd.DataFrame(means)
    d.columns = ['mean']
    d['stdev'] = [np.sqrt(p.T @ cov @ p) for p in ports]
    d['wt1'] = np.linspace(0,1,101)
    d['wt2'] = 1 - d.wt1
    d['msize'] = 1
    d['msize'] = np.where(d.wt1.isin([0,1]), 15, d.msize)
    d['correlation'] = corr
    df = pd.concat((df,d))
  df = df.round(2)
  d = {'wt1':True,'wt2':True,'correlation':False,'stdev':True,'mean':True,'msize':False}
  fig = px.line(df,x='stdev',y='mean',animation_frame='correlation',hover_data=d,markers=True)
  fig.layout.xaxis['title'] = 'Standard Deviation (%)'
  fig.layout.yaxis['title'] = 'Expected Return (%)'
  fig.update_xaxes(range=[0,35])
  fig.update_layout(margin=dict(l=30, r=20, t=20, b=200),paper_bgcolor="LightSteelBlue")
  fig['layout'].pop('updatemenus')
  fig.update_traces(marker = dict(size=df.msize))
  fig.update_xaxes(title_font_size=16)
  fig.update_yaxes(title_font_size=16)
  fig.update_layout(font_size=14)
  fig.layout.template='plotly_dark'
  return fig

app.run_server(mode='inline')

<IPython.core.display.Javascript object>

In [22]:
#@title Diversification example (stocks have 30% std devs and same correlations)
import plotly.express as px
out = None
for corr in np.linspace(0,1,101) :
  d = pd.DataFrame(dtype=float,index=range(1,301),columns=['correlation','stdev'])
  d['stdev'] = [30*np.sqrt((1+(n-1)*corr)/n) for n in range(1,301)]
  d['correlation'] = corr
  out = pd.concat((out,d))
out.reset_index(inplace=True)
out.columns=['n','correlation','stdev']
out['stdev'] = out.stdev.round(2)
fig = px.line(out,x='n',y='stdev',animation_frame='correlation')
fig.layout.xaxis['title'] = 'Number of Stocks'
fig.layout.yaxis['title'] = 'Portfolio Standard Deviation (%)'
fig.update_layout(margin=dict(l=30, r=20, t=20, b=200),paper_bgcolor="LightSteelBlue")
fig['layout'].pop('updatemenus')
fig.update_xaxes(title_font_size=16)
fig.update_yaxes(title_font_size=16,range=[0,32])
fig.update_layout(font_size=14)
fig.layout.template='plotly_dark'
fig.show()

In [ ]:
#@title Random portfolios of three assets (no shorts, fully invested)

import pandas as pd
import numpy as np
    
try :
  from jupyter_dash import JupyterDash
  from dash import dcc, html, Output, Input
except :
  !pip install jupyter-dash
  from jupyter_dash import JupyterDash
  from dash import dcc, html, Output, Input

app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Dropdown(id='rp1_dropdown',
                 value=6,
                 options=[{'label': 'Expected return of stock 1 = ' + str(rp1)+'%', 'value': rp1}
                          for rp1 in range(4,18,2)]),
    dcc.Dropdown(id='rp2_dropdown',
                 value=12,
                 options=[{'label': 'Expected return of stock 2 = ' + str(rp2)+'%', 'value': rp2}
                          for rp2 in range(4,18,2)]),
    dcc.Dropdown(id='rp3_dropdown',
                 value=14,
                 options=[{'label': 'Expected return of stock 2 = ' + str(rp3)+'%', 'value': rp3}
                          for rp3 in range(4,18,2)]),
    dcc.Dropdown(id='sd1_dropdown',
                 value=20,
                 options=[{'label': 'Standard deviation of stock 1 = ' + str(sd1)+'%', 'value': sd1}
                          for sd1 in range(15,45,5)]),
    dcc.Dropdown(id='sd2_dropdown',
                 value=30,
                 options=[{'label': 'Standard deviation of stock 2 = ' + str(sd2)+'%', 'value': sd2}
                          for sd2 in range(15,45,5)]),
    dcc.Dropdown(id='sd3_dropdown',
                 value=40,
                 options=[{'label': 'Standard deviation of stock 3 = ' + str(sd3)+'%', 'value': sd3}
                          for sd3 in range(15,45,5)]),
    dcc.Dropdown(id='c12_dropdown',
                 value=20,
                 options=[{'label': 'Correlation of stock 1 with stock 2 = ' + str(c12)+'%', 'value': c12}
                          for c12 in range(0,60,10)]),
    dcc.Dropdown(id='c13_dropdown',
                 value=30,
                 options=[{'label': 'Correlation of stock 1 with stock 3 = ' + str(c13)+'%', 'value': c13}
                          for c13 in range(0,60,10)]),
    dcc.Dropdown(id='c23_dropdown',
                 value=40,
                 options=[{'label': 'Correlation of stock 2 with stock 3 = ' + str(c23)+'%', 'value': c23}
                          for c23 in range(0,60,10)]),
    dcc.Graph(id='myfig'),
])

from scipy.stats import uniform
def random_wts(num) :
    w = uniform.rvs(0,1,num)
    return w/w.sum()
ports = [random_wts(3) for i in range(2000)]

@app.callback(Output('myfig', 'figure'),
              Input('rp1_dropdown', 'value'),
              Input('rp2_dropdown', 'value'),
              Input('rp3_dropdown', 'value'),
              Input('sd1_dropdown', 'value'),
              Input('sd2_dropdown', 'value'),
              Input('sd3_dropdown', 'value'),
              Input('c12_dropdown', 'value'),
              Input('c13_dropdown', 'value'),
              Input('c23_dropdown', 'value'),
              )
def myplot2(rp1,rp2,rp3,sd1,sd2,sd3,c12,c13,c23) :
  mns = pd.Series(np.array([rp1,rp2,rp3]), index=['stock1','stock2','stock3'])
  sds = np.array([sd1,sd2,sd3])
  C = np.diag([1.0,1.0,1.0])
  C[0,1] = c12/100
  C[0,2] = c13/100
  C[1,0] = c12/100
  C[1,2] = c23/100
  C[2,0] = c13/100
  C[2,1] = c23/100
  D = np.diag(sds/100)
  C = D @ C @ D
  
  df = pd.DataFrame(dtype=float,index=range(len(ports)),\
                    columns=['mean','stdev','wt1','wt2','wt3'])
  df['mean'] = [p @ mns for p in ports]
  df['stdev'] = [100*np.sqrt(p @ C @ p) for p in ports]
  df['wt1'] = [p[0] for p in ports]
  df['wt2'] = [p[1] for p in ports]
  df['wt3'] = [p[2] for p in ports]
  for col in ['wt1','wt2','wt3'] :
    df[col] = df[col].round(2)
  d = dict(mean=False,stdev=False,wt1=True,wt2=True,wt3=True)
  fig = px.scatter(df,x='stdev',y='mean',hover_data=d)
  fig.add_scatter(x=sds,y=mns,marker={'size':14},mode='markers',hoverinfo='skip') #,hoverinfo='skip') #,hover_data=d) 
  fig.layout.xaxis['title'] = 'Standard Deviation (%)'
  fig.layout.yaxis['title'] = 'Expected Return (%)'
  fig.update_layout(margin=dict(l=30, r=20, t=20, b=30),paper_bgcolor="LightSteelBlue")
  fig.update_xaxes(title_font_size=16 ,range=[0,50])
  fig.update_yaxes(title_font_size=16 ,range=[0,20])
  fig.update_layout(font_size=14)
  return fig

app.run_server(mode='inline')


<IPython.core.display.Javascript object>

In [ ]:
#@title Optimal portfolios of three assets (no shorts, rf=0)
from cvxopt import matrix
from cvxopt.solvers import qp as Solver
from cvxopt.solvers import options as SolverOptions
SolverOptions['show_progress'] = False

def constrainedOptimal(means,cov,raver) :
    num = len(means)
    Q = matrix(raver*cov)
    p = matrix(-means.to_numpy(),(num,1))
    G = matrix(np.vstack([-np.identity(num),np.ones(num)]))
    h = matrix(np.vstack([np.zeros(num).reshape(num,1),1]))
    # A = matrix(np.ones(num).reshape(1,num))
    # b = matrix(np.ones(1).reshape(1,1))
    sol = Solver(Q,p,G,h) # ,A,b)
    return pd.Series(sol['x'],index=means.index) if sol['status']=='optimal' else pd.Series(np.nan,index=means.index)
    
# !pip install jupyter-dash
from jupyter_dash import JupyterDash
from dash import dcc, html, Output, Input

app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Dropdown(id='rp1_dropdown',
                 value=6,
                 options=[{'label': 'Expected return of stock 1 = ' + str(rp1)+'%', 'value': rp1}
                          for rp1 in range(4,18,2)]),
    dcc.Dropdown(id='rp2_dropdown',
                 value=12,
                 options=[{'label': 'Expected return of stock 2 = ' + str(rp2)+'%', 'value': rp2}
                          for rp2 in range(4,18,2)]),
    dcc.Dropdown(id='rp3_dropdown',
                 value=14,
                 options=[{'label': 'Expected return of stock 2 = ' + str(rp3)+'%', 'value': rp3}
                          for rp3 in range(4,18,2)]),
    dcc.Dropdown(id='sd1_dropdown',
                 value=20,
                 options=[{'label': 'Standard deviation of stock 1 = ' + str(sd1)+'%', 'value': sd1}
                          for sd1 in range(15,45,5)]),
    dcc.Dropdown(id='sd2_dropdown',
                 value=30,
                 options=[{'label': 'Standard deviation of stock 2 = ' + str(sd2)+'%', 'value': sd2}
                          for sd2 in range(15,45,5)]),
    dcc.Dropdown(id='sd3_dropdown',
                 value=40,
                 options=[{'label': 'Standard deviation of stock 3 = ' + str(sd3)+'%', 'value': sd3}
                          for sd3 in range(15,45,5)]),
    dcc.Dropdown(id='c12_dropdown',
                 value=20,
                 options=[{'label': 'Correlation of stock 1 with stock 2 = ' + str(c12)+'%', 'value': c12}
                          for c12 in range(0,60,10)]),
    dcc.Dropdown(id='c13_dropdown',
                 value=30,
                 options=[{'label': 'Correlation of stock 1 with stock 3 = ' + str(c13)+'%', 'value': c13}
                          for c13 in range(0,60,10)]),
    dcc.Dropdown(id='c23_dropdown',
                 value=40,
                 options=[{'label': 'Correlation of stock 2 with stock 3 = ' + str(c23)+'%', 'value': c23}
                          for c23 in range(0,60,10)]),
    dcc.Graph(id='myfig'),
])

@app.callback(Output('myfig', 'figure'),
              Input('rp1_dropdown', 'value'),
              Input('rp2_dropdown', 'value'),
              Input('rp3_dropdown', 'value'),
              Input('sd1_dropdown', 'value'),
              Input('sd2_dropdown', 'value'),
              Input('sd3_dropdown', 'value'),
              Input('c12_dropdown', 'value'),
              Input('c13_dropdown', 'value'),
              Input('c23_dropdown', 'value'),
              )
def myplot2(rp1,rp2,rp3,sd1,sd2,sd3,c12,c13,c23) :
  mns = pd.Series(np.array([rp1,rp2,rp3])/100, index=['stock1','stock2','stock3'])
  sds = np.array([sd1,sd2,sd3])/100
  C = np.diag([1.0,1.0,1.0])
  C[0,1] = c12/100
  C[0,2] = c13/100
  C[1,0] = c12/100
  C[1,2] = c23/100
  C[2,0] = c13/100
  C[2,1] = c23/100
  D = np.diag(sds)
  C = D @ C @ D
  ravers = list(np.linspace(0,2,101)) + list(np.linspace(2,10,101)) 
  df = pd.DataFrame(dtype=float,index=range(len(ravers)),columns=['raver','mean','stdev','wt1','wt2','wt3'])
  df['raver'] = ravers
  ports = [constrainedOptimal(mns,C,raver) for raver in ravers]
  df['mean']= [p@mns for p in ports]
  df['stdev'] = [np.sqrt(p @ C @ p) for p in ports]
  df['wt1'] = [p[0] for p in ports]
  df['wt2'] = [p[1] for p in ports]
  df['wt3'] = [p[2] for p in ports]
  df[['raver','wt1','wt2','wt3']] = df[['raver','wt1','wt2','wt3']].round(2)
  # fig = px.scatter(df,x='stdev',y='mean',animation_frame='Risk Aversion')
  d = dict(raver=True,mean=False,stdev=False,wt1=True,wt2=True,wt3=True)
  fig = px.line(df,x='stdev',y='mean',hover_data=d)
  d2 = pd.DataFrame(index=range(3),columns=['mean','stdev','name'])
  d2['mean'] = mns
  d2['stdev'] = sds
  d2['name'] = ['A','B','C']
  d = dict(mean=False,stdev=False)
  fig.add_scatter(x=sds,y=mns,marker={'size':12},mode='markers') # ,hover_name='name')
  fig.layout.xaxis['title'] = 'Standard Deviation (%)'
  fig.layout.yaxis['title'] = 'Expected Return (%)'
  fig.update_layout(margin=dict(l=30, r=20, t=20, b=30),paper_bgcolor="LightSteelBlue")
  fig.update_xaxes(title_font_size=16)
  fig.update_yaxes(title_font_size=16)
  fig.update_layout(font_size=14)
  return fig

app.run_server(mode='inline')


<IPython.core.display.Javascript object>

In [26]:
#@title Optimal portfolios from historical inflation-adjusted returns
import numpy as np
import pandas as pd
import statsmodels.api as sm
import plotly.express as px





file = 'http://www.stern.nyu.edu/~adamodar/pc/datasets/histretSP.xls'
sheet = 'Returns by year'
try :
  df = pd.read_excel(file,sheet_name=sheet,skiprows=16,header=[0,1])
except :
  !pip install --upgrade xlrd
  df = pd.read_excel(file,sheet_name=sheet,skiprows=16,header=[0,1])
df = df.iloc[:93].set_index(('Unnamed: 0_level_0','Year'))
df = df['Annual Real Returns on']
df = df[df.columns[1:-1]]
df.columns = ['S&P 500','T-Bills','Treasuries','Corporates']
df = df[['S&P 500','Corporates','Treasuries','T-Bills']]


from cvxopt import matrix
from cvxopt.solvers import qp as Solver
from cvxopt.solvers import options as SolverOptions
SolverOptions['show_progress'] = False

def constrainedOptimal2(means,cov,raver) :
    num = len(means)
    Q = matrix(raver*cov)
    p = matrix(-means.to_numpy(),(num,1))
    G = matrix(-np.identity(num))
    h = matrix(np.zeros(num).reshape(num,1))
    A = matrix(np.ones(num).reshape(1,num))
    b = matrix(np.ones(1).reshape(1,1))
    sol = Solver(Q,p,G,h,A,b)
    return pd.Series(sol['x'],index=means.index) if sol['status']=='optimal' else pd.Series(np.nan,index=means.index)

'''
d = pd.DataFrame(dtype=float,index=range(df.index.min(),2001),columns=df.columns)
d.index.name = 'year'
for year in d.index :
  means = df.loc[year:].mean()
  d.loc[year] = (means==means.max())*1.0
d['raver'] = 0
'''
d = None
for raver in range(1,11) :
  d2 = pd.DataFrame(dtype=float,index=range(df.index.min(),2011),columns=df.columns)
  d2.index.name = 'year'
  for year in d2.index :
    means = df.loc[year:].mean()
    cov = df.loc[year:].cov().to_numpy()
    d2.loc[year] = constrainedOptimal2(means,cov,raver)
  d2['raver'] = raver
  d = pd.concat((d,d2))

d = d.reset_index().set_index(['raver','year']).stack().to_frame().reset_index()
d.columns=['Risk Aversion','year','asset','weight']
d['weight'] *= 100
d['weight'] = d.weight.round(1)

try: 
  from jupyter_dash import JupyterDash
  from dash import dcc, html, Output, Input
except :
  !pip install jupyter-dash
  from jupyter_dash import JupyterDash
  from dash import dcc, html, Output, Input

app = JupyterDash(__name__, title='Another Plotly Plot')

app.layout = html.Div([
    dcc.Dropdown(id='anything'),
    dcc.Markdown(''' The plot shows optimal portfolios for various levels of risk
    aversion for an investor who does not short sell (all weights are nonnegative) and
    who is fully invested in these assets (weights add to 1).  The expected returns,
    standard deviations, and correlations are estimated from historical inflation-adjusted
    (real) returns for sample periods starting at various dates.  The data comes from http://www.stern.nyu.edu/~adamodar/pc/datasets/histretSP.xls. '''),
    dcc.Graph(id='myfig')
])
@app.callback(Output('myfig','figure'),Input('anything','value'))
def myplot(anything) :
  fig = px.area(d,x='year',y='weight',color='asset',animation_frame='Risk Aversion')
  fig.layout.xaxis['title'] = 'Data Start Date'
  fig.layout.yaxis['title'] = 'Portfolio Weight (%)'
  fig.layout.yaxis['range'] = [0,100]
  fig.update_layout(margin=dict(l=30, r=20, t=20, b=200),paper_bgcolor="LightSteelBlue")
  fig['layout'].pop('updatemenus')
  fig.update_xaxes(title_font_size=16)
  fig.update_yaxes(title_font_size=16)
  fig.update_layout(font_size=14)
  fig.layout.template='plotly_dark'
  return fig


app.run_server(mode='inline')

<IPython.core.display.Javascript object>

In [19]:
#@title CAPM betas and risk premia of Fama-French 48 industries
import numpy as np
import pandas as pd
import statsmodels.api as sm
import plotly.express as px
from pandas_datareader import DataReader as pdr
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

d = dict(Agric='Agriculture',Food='Food Products',Soda='Candy & Soda',Beer='Beer & Liquor',Smoke='Tobacco Products',Toys='Recreation',\
Fun='Entertainment',Books='Printing & Publishing',Hshld='Consumer Goods',Clths='Apparel',Hlth='Healthcare',MedEq='Medical Equipment',\
Drugs='Pharmaceutical Products',Chems='Chemicals',Rubbr='Rubber and Plastic Products',Txtls='Textiles',BldMt='Construction Materials',\
Cnstr='Construction',Steel='Steel Works Etc',FabPr='Fabricated Products',Mach='Machinery',ElcEq='Electrical Equipment',Autos='Automobiles & Trucks',\
Aero='Aircraft',Ships='Shipbuilding & Railroad Equipment',Guns='Defense',Gold='Precious Metals',Mines='Non-Metallic & Industrial Metal Mining',\
Coal='Coal',Oil='Petroleum & Natural Gas',Util='Utilities',Telcm='Communication',PerSv='Personal Services',BusSv='Business Services',\
Comps='Computers',Chips='Electronic Equipment',LabEq='Measuring & Control Equipment',Paper='Business Supplies',Boxes='Shipping Containers',\
Trans='Transportation',Whlsl='Wholesale',Rtail='Retail',Meals='Restaurants, Hotels, & Motels',Banks='Banking',Insur='Insurance',RlEst='Real Estate',\
Fin='Trading',Other='Almost Nothing')

ff = pdr('F-F_Research_Data_5_Factors_2x3','famafrench',start=1970)[0] 
df = pdr('48_Industry_Portfolios','famafrench',start=1970)[0]
df.columns = [x.strip() for x in df.columns]
df = df.rename(columns=d)
inds = df.columns.to_list()
df = df.join(ff,how='inner')
df[inds] = df[inds].subtract(df.RF,axis='index')

starts = map(str,[1970,1980,1990,2000])
out = None
for start in starts :
    d1 = df.loc[start+'-01':]
    d2 = pd.DataFrame(dtype=float,index=inds,columns=['rprem','beta'])
    d2['rprem'] = 12*d1[inds].mean()
    X = sm.add_constant(d1['Mkt-RF'])
    for ind in inds :
        d2.loc[ind,'beta'] = sm.OLS(d1[ind],X).fit().params['Mkt-RF']
    d2['Data Start Date'] = start
    out = pd.concat((out,d2))
out.reset_index(inplace=True)
out.columns=['industry','rprem','beta','Data Start Date']
out[['rprem','beta']] = out[['rprem','beta']].round(2)
minx = out.beta.min() - 0.05
maxx = out.beta.max() + 0.05
miny = out.rprem.min() - 1
maxy = out.rprem.max() + 1
d = {'rprem':True,'beta':True,'Data Start Date':False}

try: 
  from jupyter_dash import JupyterDash
  from dash import dcc, html, Output, Input
except :
  !pip install jupyter-dash
  from jupyter_dash import JupyterDash
  from dash import dcc, html, Output, Input

app = JupyterDash(__name__, title='Another Plotly Plot')

app.layout = html.Div([
    dcc.Dropdown(id='anything'),
    dcc.Markdown(''' The plot shows the extimated market betas and historical risk premia
    of value-weighted industry returns using the Fama-French 48 industry classification.
    The betas are estimated from monthly returns.  All data comes from Keneth French's
    data library: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html.
    '''),
    dcc.Graph(id='myfig')
])
@app.callback(Output('myfig','figure'),Input('anything','value'))
def myplot(anything) :
  fig = px.scatter(out, x="beta", y="rprem",hover_name='industry',\
                 trendline='ols',animation_frame='Data Start Date',hover_data=d)
  fig.layout.xaxis['title'] = 'Beta'
  fig.layout.yaxis['title'] = 'Risk Premium (Annualized %)'
  fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
  fig.update_layout(margin=dict(l=30, r=20, t=20, b=200),paper_bgcolor="LightSteelBlue")
  fig.update_xaxes(title_font_size=16,range=[minx,maxx])
  fig.update_yaxes(title_font_size=16,range=[miny,maxy])
  fig.update_layout(font_size=14)
  fig.layout.template='plotly_dark'
  fig['layout'].pop('updatemenus')
  return fig

app.run_server(mode='inline')


<IPython.core.display.Javascript object>

In [ ]:
#@title CAPM predictions and risk premia of Fama-French 48 industries
import numpy as np
import pandas as pd
import statsmodels.api as sm
import plotly.express as px
from pandas_datareader import DataReader as pdr
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

d = dict(Agric='Agriculture',Food='Food Products',Soda='Candy & Soda',Beer='Beer & Liquor',Smoke='Tobacco Products',Toys='Recreation',\
Fun='Entertainment',Books='Printing & Publishing',Hshld='Consumer Goods',Clths='Apparel',Hlth='Healthcare',MedEq='Medical Equipment',\
Drugs='Pharmaceutical Products',Chems='Chemicals',Rubbr='Rubber and Plastic Products',Txtls='Textiles',BldMt='Construction Materials',\
Cnstr='Construction',Steel='Steel Works Etc',FabPr='Fabricated Products',Mach='Machinery',ElcEq='Electrical Equipment',Autos='Automobiles & Trucks',\
Aero='Aircraft',Ships='Shipbuilding & Railroad Equipment',Guns='Defense',Gold='Precious Metals',Mines='Non-Metallic & Industrial Metal Mining',\
Coal='Coal',Oil='Petroleum & Natural Gas',Util='Utilities',Telcm='Communication',PerSv='Personal Services',BusSv='Business Services',\
Comps='Computers',Chips='Electronic Equipment',LabEq='Measuring & Control Equipment',Paper='Business Supplies',Boxes='Shipping Containers',\
Trans='Transportation',Whlsl='Wholesale',Rtail='Retail',Meals='Restaurants, Hotels, & Motels',Banks='Banking',Insur='Insurance',RlEst='Real Estate',\
Fin='Trading',Other='Almost Nothing')

ff = pdr('F-F_Research_Data_5_Factors_2x3','famafrench',start=1970)[0] 
df = pdr('48_Industry_Portfolios','famafrench',start=1970)[0]
df.columns = [x.strip() for x in df.columns]
df = df.rename(columns=d)
inds = df.columns.to_list()
df = df.join(ff,how='inner')
df[inds] = df[inds].subtract(df.RF,axis='index')

starts = map(str,[1970,1980,1990,2000])
out = None
for start in starts :
    d1 = df.loc[start+'-01':]
    d2 = pd.DataFrame(dtype=float,index=inds,columns=['actual','predicted'])
    d2['actual'] = 12*d1[inds].mean()
    X = sm.add_constant(d1['Mkt-RF'])
    mprem = 12*d1['Mkt-RF'].mean()
    for ind in inds :
        beta = sm.OLS(d1[ind],X).fit().params['Mkt-RF']
        d2.loc[ind,'predicted'] = beta*mprem
    d2['Data Start Date'] = start
    out = pd.concat((out,d2))
out.reset_index(inplace=True)
out.columns=['industry','actual','predicted','Data Start Date']
out[['actual','predicted']] = out[['actual','predicted']].round(2)
minx = out.predicted.min() - 0.5
maxx = out.predicted.max() + 0.5
miny = out.actual.min() - 1
maxy = out.actual.max() + 1
d = {'actual':True,'predicted':True,'Data Start Date':False}
fig = px.scatter(out, x="predicted", y="actual",hover_name='industry',\
                 trendline='ols',animation_frame='Data Start Date',hover_data=d)
fig.layout.xaxis['title'] = 'Predicted Risk Premium (Annualized %)'
fig.layout.yaxis['title'] = 'Actual Risk Premium (Annualized %)'
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.update_layout(margin=dict(l=30, r=20, t=20, b=200),paper_bgcolor="LightSteelBlue")
fig.update_xaxes(title_font_size=16,range=[minx,maxx])
fig.update_yaxes(title_font_size=16,range=[miny,maxy])
fig.update_layout(font_size=14)
fig['layout'].pop('updatemenus')
fig.show()

In [31]:
#@title Fama-French predictions and risk premia of Fama-French 48 industries
import numpy as np
import pandas as pd
import statsmodels.api as sm
import plotly.express as px
from pandas_datareader import DataReader as pdr
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

d = dict(Agric='Agriculture',Food='Food Products',Soda='Candy & Soda',Beer='Beer & Liquor',Smoke='Tobacco Products',Toys='Recreation',\
Fun='Entertainment',Books='Printing & Publishing',Hshld='Consumer Goods',Clths='Apparel',Hlth='Healthcare',MedEq='Medical Equipment',\
Drugs='Pharmaceutical Products',Chems='Chemicals',Rubbr='Rubber and Plastic Products',Txtls='Textiles',BldMt='Construction Materials',\
Cnstr='Construction',Steel='Steel Works Etc',FabPr='Fabricated Products',Mach='Machinery',ElcEq='Electrical Equipment',Autos='Automobiles & Trucks',\
Aero='Aircraft',Ships='Shipbuilding & Railroad Equipment',Guns='Defense',Gold='Precious Metals',Mines='Non-Metallic & Industrial Metal Mining',\
Coal='Coal',Oil='Petroleum & Natural Gas',Util='Utilities',Telcm='Communication',PerSv='Personal Services',BusSv='Business Services',\
Comps='Computers',Chips='Electronic Equipment',LabEq='Measuring & Control Equipment',Paper='Business Supplies',Boxes='Shipping Containers',\
Trans='Transportation',Whlsl='Wholesale',Rtail='Retail',Meals='Restaurants, Hotels, & Motels',Banks='Banking',Insur='Insurance',RlEst='Real Estate',\
Fin='Trading',Other='Almost Nothing')

ff = pdr('F-F_Research_Data_5_Factors_2x3','famafrench',start=1970)[0] 
df = pdr('48_Industry_Portfolios','famafrench',start=1970)[0]
df.columns = [x.strip() for x in df.columns]
df = df.rename(columns=d)
inds = df.columns.to_list()
df = df.join(ff,how='inner')
df[inds] = df[inds].subtract(df.RF,axis='index')

starts = map(str,[1970,1980,1990,2000])
out = None
for start in starts :
    d1 = df.loc[start+'-01':]
    d2 = pd.DataFrame(dtype=float,index=inds,columns=['actual','predicted'])
    d2['actual'] = 12*d1[inds].mean()
    X = sm.add_constant(d1[['Mkt-RF','SMB','HML','CMA','RMW']])
    fprem = 12*d1[['Mkt-RF','SMB','HML','CMA','RMW']].mean()
    for ind in inds :
        betas = sm.OLS(d1[ind],X).fit().params[['Mkt-RF','SMB','HML','CMA','RMW']]
        d2.loc[ind,'predicted'] = betas @ fprem
    d2['Data Start Date'] = start
    out = pd.concat((out,d2))
out.reset_index(inplace=True)
out.columns=['industry','actual','predicted','Data Start Date']
out[['actual','predicted']] = out[['actual','predicted']].round(2)
minx = out.predicted.min() - 0.5
maxx = out.predicted.max() + 0.5
miny = out.actual.min() - 1
maxy = out.actual.max() + 1
d = {'actual':True,'predicted':True,'Data Start Date':False}

try: 
  from jupyter_dash import JupyterDash
  from dash import dcc, html, Output, Input
except :
  !pip install jupyter-dash
  from jupyter_dash import JupyterDash
  from dash import dcc, html, Output, Input

app = JupyterDash(__name__, title='Another Plotly Plot')

app.layout = html.Div([
    dcc.Dropdown(id='anything'),
    dcc.Markdown(''' The plot shows the predicted risk premia and historical risk premia
    of value-weighted industry returns using the Fama-French 48 industry classification.
    The predicted risk premia are based on the five-factor Fama-French model and equal
    the sum of betas multiplied by the historical factor risk premia.  Betas are estimated
    from monthly returns.  All data comes from Keneth French's
    data library: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html.
    '''),
    dcc.Graph(id='myfig')
])
@app.callback(Output('myfig','figure'),Input('anything','value'))
def myplot(anything) :
  fig = px.scatter(out, x="predicted", y="actual",hover_name='industry',\
                 trendline='ols',animation_frame='Data Start Date',hover_data=d)
  fig.layout.xaxis['title'] = 'Predicted Risk Premium (Annualized %)'
  fig.layout.yaxis['title'] = 'Actual Risk Premium (Annualized %)'
  fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
  fig.update_layout(margin=dict(l=30, r=30, t=50, b=200),paper_bgcolor="LightSteelBlue")
  fig.update_xaxes(title_font_size=16,range=[minx,maxx])
  fig.update_yaxes(title_font_size=16,range=[miny,maxy])
  fig.update_layout(font_size=14)
  fig['layout'].pop('updatemenus')
  fig.layout.template='plotly_dark'
  return fig

app.run_server(mode='inline')


<IPython.core.display.Javascript object>